### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [19]:
from sqp import *
np.set_printoptions(suppress=True)

The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [20]:
def run_sqp(test_string, lmbd):
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l) + 1e-10, np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    x0 = pbm.TransitionMatrix.reshape(-1)
    x0 = np.hstack((x0, np.dot(C, x0) + 1)).copy()
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-14, 'disp': True},
            bounds=bounds
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [21]:
M, StateTransMatrix = run_sqp("ABACAABCBABBCC", 0.1)
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)

[2024-02-06 00:46:58.946] Objective Fxn Value = 49.22179040210122
[2024-02-06 00:46:58.947] Objective Fxn Value = 15.651150561123762
[2024-02-06 00:46:58.948] Objective Fxn Value = 14.281959758296061
[2024-02-06 00:46:58.948] Objective Fxn Value = 14.281959758285451
Optimization terminated successfully    (Exit mode 0)
            Current function value: 14.281959758285451
            Iterations: 8
            Function evaluations: 4
            Gradient evaluations: 4
The original MLE estimated transition matrix is: 


array([[0.2       , 0.6       , 0.2       ],
       [0.4       , 0.2       , 0.4       ],
       [0.33333333, 0.33333333, 0.33333333]])

The New Transiiton Matrix after the SQP is: 


array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [24]:
M, StateTransMatrix = run_sqp("ABACAABCBABBCC", 0.03)
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)

[2024-02-06 00:47:29.530] Objective Fxn Value = 49.29179040210122
[2024-02-06 00:47:29.531] Objective Fxn Value = 14.138992470057689
[2024-02-06 00:47:29.532] Objective Fxn Value = 14.281959749618947
[2024-02-06 00:47:29.532] Objective Fxn Value = 14.040532727997737
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.281959762953077
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.068523105389138
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.046696104983715
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.04233277436979
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.041169972406864
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.04079056475471
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.040646148727511
[2024-02-06 00:47:29.533] Objective Fxn Value = 14.04058503130019
[2024-02-06 00:47:29.534] Objective Fxn Value = 14.040557453537886
[2024-02-06 00:47:29.534] Objective Fxn Value = 14.040544564161708
[2024-02-06 00:47:29.534] Objective Fxn Value = 14.040538429276674

array([[0.2       , 0.6       , 0.2       ],
       [0.4       , 0.2       , 0.4       ],
       [0.33333333, 0.33333333, 0.33333333]])

The New Transiiton Matrix after the SQP is: 


array([[0.26979075, 0.46239846, 0.26781557],
       [0.36161883, 0.28086377, 0.3575174 ],
       [0.33333333, 0.33333333, 0.33333333]])